In [1]:
import sys
sys.path.append('../..')
from utilities.get_data import get_historical_from_db
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
import ta
import pandas_ta as pda
import matplotlib.pyplot as plt

In [2]:
# --- Load Data ---
pair = "ETH/USDT"
tf = "1h"

df = get_historical_from_db(ccxt.binance(),pair,tf,path="../../database/")
df

,open,high,low,close,volume
date,,,,,
2017-08-17 04:00:00,301.13,302.57,298.00,301.61,125.66877
2017-08-17 05:00:00,301.61,303.28,300.00,303.10,377.67246
2017-08-17 06:00:00,302.40,304.44,301.90,302.68,303.86672
2017-08-17 07:00:00,302.68,307.96,302.60,307.96,754.74510
2017-08-17 08:00:00,307.95,309.97,307.00,308.62,150.75029
...,...,...,...,...,...
2022-05-01 08:00:00,2766.40,2782.00,2765.18,2778.31,12854.70750
2022-05-01 09:00:00,2778.31,2780.64,2756.80,2768.31,8475.55070
2022-05-01 10:00:00,2768.32,2775.87,2761.00,2766.42,8231.25800


In [4]:
df['LAST_ATH'] = df['close'].cummax()

print("Indicators loaded 100%")
df

Indicators loaded 100%


,open,high,low,close,volume,LAST_ATH
date,,,,,,
2017-08-17 04:00:00,301.13,302.57,298.00,301.61,125.66877,301.61
2017-08-17 05:00:00,301.61,303.28,300.00,303.10,377.67246,303.10
2017-08-17 06:00:00,302.40,304.44,301.90,302.68,303.86672,303.10
2017-08-17 07:00:00,302.68,307.96,302.60,307.96,754.74510,307.96
2017-08-17 08:00:00,307.95,309.97,307.00,308.62,150.75029,308.62
...,...,...,...,...,...,...
2022-05-01 08:00:00,2766.40,2782.00,2765.18,2778.31,12854.70750,4846.71
2022-05-01 09:00:00,2778.31,2780.64,2756.80,2768.31,8475.55070,4846.71
2022-05-01 10:00:00,2768.32,2775.87,2761.00,2766.42,8231.25800,4846.71


In [5]:
dfTest = df[:]
weeklyAmount = 30
takerFee = 0.0007
buyAmount = weeklyAmount
mediumBuy = 0
totalInvest = 0
btcWallet = 0

for index, row in dfTest.iterrows():
    totalInvest += buyAmount
    buyBTC = buyAmount / row['close']
    btcWallet += buyBTC - takerFee * buyBTC
    mediumBuy += 1

resultInDollar = btcWallet * dfTest.iloc[-1]['close']
perfInPct = (resultInDollar - totalInvest)/totalInvest
buyAndHoldPerf = (dfTest.iloc[-1]['close'] - dfTest.iloc[0]['close'])/dfTest.iloc[0]['close']
print('Buy',mediumBuy,'time',weeklyAmount,'$')   
print('Total invest', totalInvest, '$')
print('Final wallet', round(btcWallet,3), 'BTC')
print('Final wallet equivalent', round(resultInDollar,2), '$')
print('Performance',round(perfInPct*100,2), '%')
print('Buy and Hold performance', round(buyAndHoldPerf*100,2), '%')

Buy 41115 time 30 $
Total invest 1233450 $
Final wallet 3895.551 BTC
Final wallet equivalent 10759512.99 $
Performance 772.31 %
Buy and Hold performance 815.75 %


In [6]:
dfTest = df[:]
weeklyAmount = 30
takerFee = 0.0007
buyAmount = 0
bigBuy = 0
mediumBuy = 0
lowBuy = 0
totalEntry = 0
totalInvest = 0
btcWallet = 0

for index, row in dfTest.iterrows():
    if row['close'] <= 0.5 * row['LAST_ATH']:
        buyAmount = 2 * weeklyAmount
        bigBuy+=1
    elif row['close'] > 0.5 * row['LAST_ATH'] and row['close'] <= 0.8 * row['LAST_ATH']:
        mediumBuy+=1
        buyAmount = 1 * weeklyAmount
    elif row['close'] > 0.8 * row['LAST_ATH']:
        lowBuy+=1
        buyAmount = 0.5 * weeklyAmount
        # buyAmount = 0
    totalInvest += buyAmount
    buyBTC = buyAmount / row['close']
    btcWallet += buyBTC - takerFee * buyBTC
    totalEntry += 1

resultInDollar = btcWallet * dfTest.iloc[-1]['close']
perfInPct = (resultInDollar - totalInvest)/totalInvest
buyAndHoldPerf = (dfTest.iloc[-1]['close'] - dfTest.iloc[0]['close'])/dfTest.iloc[0]['close']
print('Buy',bigBuy,'time',2*weeklyAmount,'$')   
print('Buy',mediumBuy,'time',1*weeklyAmount,'$')   
print('Buy',lowBuy,'time',0.5*weeklyAmount,'$')   
print('Total invest', totalInvest, '$')
print('Final wallet', round(btcWallet,3), 'BTC')
print('Final wallet equivalent', round(resultInDollar,2), '$')
print('Performance',round(perfInPct*100,2), '%')
print('Buy and Hold performance', round(buyAndHoldPerf*100,2), '%')

Buy 25064 time 60 $
Buy 9011 time 30 $
Buy 7040 time 15.0 $
Total invest 1879770.0 $
Final wallet 7220.098 BTC
Final wallet equivalent 19941909.88 $
Performance 960.87 %
Buy and Hold performance 815.75 %
